In [1]:
! pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 6.8 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619430 sha256=120aa0380933cf6f7a9367a43482297776d0b7db0fb8c3dcd0991110ea6c873d
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


## Surprise를 이용한 추천 시스템 구축

In [2]:
from surprise import SVD
from surprise import Dataset 
from surprise import accuracy 
from surprise.model_selection import train_test_split

In [3]:
data = Dataset.load_builtin('ml-100k') 
# 수행 시마다 동일하게 데이터를 분할하기 위해 random_state 값 부여
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [4]:
algo = SVD()
algo.fit(trainset)

In [5]:
predictions = algo.test( testset )
print('prediction type :',type(predictions), ' size:',len(predictions))
print('prediction 결과의 최초 5개 추출')
predictions[:5]

prediction type : <class 'list'>  size: 25000
prediction 결과의 최초 5개 추출


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.558282649979794, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.8587951893424295, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.842464602130221, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.676643177003125, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.5800203916298803, details={'was_impossible': False})]

In [6]:
[ (pred.uid, pred.iid, pred.est) for pred in predictions[:3] ]

[('120', '282', 3.558282649979794),
 ('882', '291', 3.8587951893424295),
 ('535', '507', 3.842464602130221)]

In [8]:
# 사용자 아이디, 아이템 아이디는 문자열로 입력해야 함. 
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.49   {'was_impossible': False}


In [9]:
accuracy.rmse(predictions)

RMSE: 0.9464


0.946409526104768

## Surprise 주요 모듈 소개

* Dataset

In [11]:
from google.colab import drive
import pandas as pd

drive.mount("/content/gdrive", force_remount=True)
default_path='/content/gdrive/MyDrive/ESAA 2학기/데이터/'

Mounted at /content/gdrive


In [12]:
import pandas as pd

ratings = pd.read_csv(default_path + 'ratings.csv')
# ratings_noh.csv 파일로 unload 시 index 와 header를 모두 제거한 새로운 파일 생성.  
ratings.to_csv(default_path + 'ratings_noh.csv', index=False, header=False)

In [14]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data=Dataset.load_from_file(default_path + 'ratings_noh.csv',reader=reader)

In [15]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

# 수행시마다 동일한 결과 도출을 위해 random_state 설정 
algo = SVD(n_factors=50, random_state=0)

# 학습 데이터 세트로 학습 후 테스트 데이터 세트로 평점 예측 후 RMSE 평가
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

* 판다스 DataFrame에서 Surprise 데이터 세트로 로딩

In [16]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv(default_path + 'ratings.csv') 
reader = Reader(rating_scale=(0.5, 5.0))

# ratings DataFrame 에서 컬럼은 사용자 아이디, 아이템 아이디, 평점 순서를 지켜야 합니다. 
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset) 
predictions = algo.test( testset )
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

## Surprise 추천 알고리즘 시스템

In [17]:
from surprise.model_selection import cross_validate 

# Pandas DataFrame에서 Surprise Dataset으로 데이터 로딩 
ratings = pd.read_csv(default_path + 'ratings.csv') # reading data in pandas df
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algo = SVD(random_state=0) 
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8734  0.8724  0.8772  0.8689  0.8718  0.8727  0.0027  
MAE (testset)     0.6728  0.6687  0.6725  0.6692  0.6686  0.6704  0.0019  
Fit time          4.89    4.82    4.93    4.88    4.92    4.89    0.04    
Test time         0.26    0.15    0.15    0.30    0.15    0.20    0.06    


{'fit_time': (4.891610383987427,
  4.818360090255737,
  4.934629917144775,
  4.88458251953125,
  4.917568683624268),
 'test_mae': array([0.67276737, 0.66874429, 0.67245046, 0.66915989, 0.66863491]),
 'test_rmse': array([0.87344805, 0.87239882, 0.87720462, 0.86891222, 0.87177765]),
 'test_time': (0.25775885581970215,
  0.15150737762451172,
  0.15157318115234375,
  0.29598259925842285,
  0.14965105056762695)}

In [18]:
from surprise.model_selection import GridSearchCV

# 최적화할 파라미터들을 딕셔너리 형태로 지정. 
param_grid = {'n_epochs': [20, 40, 60], 'n_factors': [50, 100, 200] }

# CV를 3개 폴드 세트로 지정, 성능 평가는 rmse, mse 로 수행 하도록 GridSearchCV 구성
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# 최고 RMSE Evaluation 점수와 그때의 하이퍼 파라미터
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8771826702000346
{'n_epochs': 20, 'n_factors': 50}
